## Import Libraries

In [1]:
from mpi4py import MPI
import numpy as np

## Basic setup

In [2]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

## Simple Sequential Quicksort

In [3]:
def sequential_quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        left = [x for x in arr[1:] if x <= pivot]
        right = [x for x in arr[1:] if x > pivot]
        return sequential_quicksort(left) + [pivot] + sequential_quicksort(right)

## Parallel Sequential Quicksort

In [4]:
def parallel_quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        left = [x for x in arr[1:] if x <= pivot]
        right = [x for x in arr[1:] if x > pivot]

        # Distribute sub-arrays to different processors
        left_sub_arr = np.array_split(left, size)
        right_sub_arr = np.array_split(right, size)

        # Scatter sub-arrays to different processors
        left_sub_arr = comm.scatter(left_sub_arr, root=0)
        right_sub_arr = comm.scatter(right_sub_arr, root=0)
     # Perform sequential Quicksort on sub-arrays
        left_sub_arr = sequential_quicksort(left_sub_arr)
        right_sub_arr = sequential_quicksort(right_sub_arr)
        # Gather sorted sub-arrays from different processors
        sorted_left = comm.gather(left_sub_arr, root=0)
        sorted_right = comm.gather(right_sub_arr, root=0)
        if rank == 0:
            # Merge sorted sub-arrays
            sorted_left = np.concatenate(sorted_left)
            sorted_right = np.concatenate(sorted_right)
            return np.concatenate((sorted_left, [pivot], sorted_right))

## Main Code

In [ ]:
if rank == 0:
    # Generate random array of integers
    arr = np.random.randint(0, 10000, size=1000000)

    # Measure time required for sequential Quicksort
    seq_start_time = MPI.Wtime()
    sorted_arr_seq = sequential_quicksort(arr)
    seq_end_time = MPI.Wtime()
    seq_time = seq_end_time - seq_start_time

    # Measure time required for parallel Quicksort
    par_start_time = MPI.Wtime()
    sorted_arr_par = parallel_quicksort(arr)
    par_end_time = MPI.Wtime()
    par_time = par_end_time - par_start_time

    # Check if sorted arrays match
    assert np.array_equal(sorted_arr_seq, sorted_arr_par)

    # Print time required for sequential Quicksort and parallel Quicksort
    print(f"Time required for sequential Quicksort: {seq_time:.6f} seconds")
    print(f"Time required for parallel Quicksort: {par_time:.6f} seconds")
    
MPI.Finalize()